In [1]:
from openpyxl import load_workbook
wb = load_workbook(filename = 'game_data/kukui.xlsx')

/Users/chiraag/anaconda/lib/python2.7/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [2]:
class BasicMove(object):
    def __init__(self, row):
        self.mid = int(row[0].value)
        self.name = row[1].value
        self.ptype = row[2].value
        self.power = int(row[3].value)
        self.stamina_loss_scalar = row[4].value
        self.duration = int(row[5].value) # in ms
        self.damage_window = int(row[6].value) # in ms
        self.energy = int(row[7].value)
        self.eps = row[8].value
        self.dps = row[9].value
        
    def __str__(self):
        return '%d %s: %d %d %d %.2g %.2g (%s)' % (self.mid, self.name, self.power, \
                self.duration, self.energy, self.eps, self.dps, self.ptype)

sheet = wb.get_sheet_by_name('Basic Moves')
basic_moves = { row[1].value: BasicMove(row) for i, row in enumerate(sheet.iter_rows()) if 0 < i < 43}
# for m in basic_moves:
#    print basic_moves[m]

In [3]:
class ChargedMove(object):
    def __init__(self, row):
        self.mid = int(row[0].value)
        self.name = row[1].value
        self.ptype = row[2].value
        self.power = int(row[3].value)
        self.stamina_loss_scalar = row[4].value
        self.heal_scalar = row[5].value
        self.duration = int(row[6].value) # in ms
        self.dodge_window = int(row[7].value) # in ms
        self.crit = row[8].value
        self.cost = int(row[9].value)
        
    def __str__(self):
        return '%d %s: %d %d %d %d %.2g (%s)' % (self.mid, self.name, self.power, \
                self.duration, self.dodge_window, self.cost, self.crit, self.ptype)

sheet = wb.get_sheet_by_name('Charged Moves')
charged_moves = { row[1].value: ChargedMove(row) for i, row in enumerate(sheet.iter_rows()) if 0 < i < 96}

# for m in charged_moves:
#    print charged_moves[m]

In [4]:
class Species(object):
    def __init__(self, row):
        self.name = row[0].value
        self.pid = int(row[1].value)
        self.hp_ratio = int(row[2].value)
        self.attack_ratio = int(row[3].value)
        self.defense_ratio = int(row[4].value)
        # self.cp_limits = [int(row[5].value), int(row[6].value)]
        self.ptypes = []
        self.family = self.name
        self.evolutions = {}
        
    def __str__(self):
        return '%03d %s: %d %d %d' % (self.pid, self.name, self.hp_ratio, \
                self.attack_ratio, self.defense_ratio)
    
sheet = wb.get_sheet_by_name('Species Data')
species = { row[0].value: Species(row) for i, row in enumerate(sheet.iter_rows()) if 0 < i < 152}

sheet = wb.get_sheet_by_name('Showing Work')
for i, row in enumerate(sheet.iter_rows()):
    if 0 < i < 896:
        species[row[6].value].ptypes = [row[col].value for col in [7, 8] if row[col].value is not None]

# print species['Pidgeot'].ptypes
# for s in species:
#   print species[s]

In [5]:
class Pokemon(object):
    def __init__(self):
        self.name = None
        self.cp = 0
        self.hp = 0
        self.powerup_dust = 0
        self.species = None
        self.level = 0
        self.iv = (0,0,0)
        self.basic_move = None
        self.charged_move = None

In [6]:
wb = load_workbook(filename = 'game_data/evolution.xlsx')
for row in wb.active.iter_rows():
    sp_from = row[1].value
    candy, rest = row[2].value.split(' ', 1)
    candy = int(candy)
    sp_base = rest.split(' Candy')[0]
    sp_to = row[4].value
    species[sp_from].evolutions[sp_to] = candy
    species[sp_from].family = sp_base
    species[sp_to].family = sp_base
    # print sp_from, sp_to, candy, sp_base

In [7]:
import numpy as np
cpm = np.array([ 0.094     ,  0.16639787,  0.21573247,  0.25572005,  0.29024988,
        0.3210876 ,  0.34921268,  0.37523559,  0.39956728,  0.42250001,
        0.44310755,  0.46279839,  0.48168495,  0.49985844,  0.51739395,
        0.53435433,  0.55079269,  0.56675452,  0.58227891,  0.59740001,
        0.61215729,  0.62656713,  0.64065295,  0.65443563,  0.667934  ,
        0.68116492,  0.69414365,  0.70688421,  0.71939909,  0.7317    ,
        0.73776948,  0.74378943,  0.74976104,  0.75568551,  0.76156384,
        0.76739717,  0.7731865 ,  0.77893275,  0.78463697,  0.79030001])